In [116]:
import torch
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plt

# a quadrilateral has four corners (x1, y1), (x2, y2), (x3, y3), (x4, y4)
# We want optimize the area of the quadrilateral while constraining the perimeter to 100 units

x1 = Variable(torch.randn(1), requires_grad=True)
y1 = Variable(torch.randn(1), requires_grad=True)
x2 = Variable(torch.randn(1), requires_grad=True)
y2 = Variable(torch.randn(1), requires_grad=True)
x3 = Variable(torch.randn(1), requires_grad=True)
y3 = Variable(torch.randn(1), requires_grad=True)
x4 = Variable(torch.randn(1), requires_grad=True)
y4 = Variable(torch.randn(1), requires_grad=True)

# Define the function for the area 
def compute_area(x1, y1, x2, y2, x3, y3, x4, y4):
    # Area = 0.5 * (d1 *d2)
    # where d1 = ((x1 - x3),(y1 - y3))
    # where d2 = ((x2 - x4),(y2 - y4))    
    return (0.5 * (torch.sqrt((x1 - x3)**2+(y1 - y3)**2))* 0.5 *(torch.sqrt((x2 - x4)**2+(y2 - y4)**2)))

# Define the function for the perimeter 
def compute_perimeter(x1, y1, x2, y2, x3, y3, x4, y4):
    # Perimeter = sum of the lengths of its sides
    s1 = torch.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    s2 = torch.sqrt((x3 - x2)**2 + (y3 - y2)**2)
    s3 = torch.sqrt((x4 - x3)**2 + (y4 - y3)**2)
    s4 = torch.sqrt((x1 - x4)**2 + (y1 - y4)**2)
    return s1 + s2 + s3 + s4

# Define the optimizer (we are using stochastic gradient descent)
optimizer = optim.RMSprop([x1, y1, x2, y2, x3, y3, x4, y4], lr=.02)

#Adam
#SGD

# Number of steps in the optomisation
steps = 1000

# perimeter constraint...limmited to 100 units
max_perimeter = 100

#loss_history = [] #collecting the value of the loss function for each interation 

# Optimization loop
for step in range(steps):
    # Compute area
    area = compute_area(x1, y1, x2, y2, x3, y3, x4, y4)
    
    # Compute perimeter
    perimeter = compute_perimeter(x1, y1, x2, y2, x3, y3, x4, y4)
    
    # apply a penalty if peremter of 100 units is exceeded (not really confindent in this step and the next step)
    perimeter_penalty = torch.max(torch.tensor(0.), ((perimeter)**2 - (max_perimeter)**2)) #hmmmm
    
    # loss function
    loss = (-area + perimeter_penalty**1)  # hmmmm
    
    # Zero gradients
    optimizer.zero_grad()
    
    # Compute gradients
    loss.backward()
    
    # Update parameters
    optimizer.step()

    if step % 100 == 0:
        print(f'Step {step}: loss_history = {loss.item()}') # Prints a value for loss function once every hundred iterations

# optomised coords
optimized_x1 = x1.item()
optimized_y1 = y1.item()
optimized_x2 = x2.item()
optimized_y2 = y2.item()
optimized_x3 = x3.item()
optimized_y3 = y3.item()
optimized_x4 = x4.item()
optimized_y4 = y4.item()


# Print the optimized coords, area and perimeter (should be a square)
print("Optimized Coordinates:")
print("Vertex 1: ({}, {})".format(optimized_x1, optimized_y1))
print("Vertex 2: ({}, {})".format(optimized_x2, optimized_y2))
print("Vertex 3: ({}, {})".format(optimized_x3, optimized_y3))
print("Vertex 4: ({}, {})".format(optimized_x4, optimized_y4))
print("Optimized Perimeter: {:.4f}".format(compute_perimeter(x1, y1, x2, y2, x3, y3, x4, y4).item()))
print("Optimized Area: {:.4f}".format(compute_area(x1, y1, x2, y2, x3, y3, x4, y4).item()))
print("Theoretical maximum = 625")



Step 0: loss_history = -1.613047480583191
Step 100: loss_history = -75.54415130615234
Step 200: loss_history = -165.25042724609375
Step 300: loss_history = -272.4236145019531
Step 400: loss_history = -397.2742004394531
Step 500: loss_history = -539.8086547851562
Step 600: loss_history = -609.9346923828125
Step 700: loss_history = -621.4909057617188
Step 800: loss_history = -620.7418823242188
Step 900: loss_history = -619.7544555664062
Optimized Coordinates:
Vertex 1: (-17.152851104736328, 17.521093368530273)
Vertex 2: (-17.172231674194336, 17.527090072631836)
Vertex 3: (17.930219650268555, -17.87075424194336)
Vertex 4: (17.98978614807129, -17.957000732421875)
Optimized Perimeter: 99.9137
Optimized Area: 622.3597
Theoretical maximum = 625
